<a href="https://colab.research.google.com/github/GeoTurkey/GMT_COURSES/blob/main/AAPL_STOCK_PRICE_PyTorch_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN Exercises


In the exercises below you'll be asked to do the following:
* Perform standard imports, load & plot the dataset (code provided)
* Prepare data for an LSTM model
* Define the LSTM model, loss and optimization functions
* Train the model
* Evaluate the model on test data
* OPTIONAL: Plot the results

## Perform standard imports, load and plot the dataset
Run the cells below to load the libraries needed for this exercise and the Energy Production dataset, and to plot the data.

In [ ]:
# RUN THIS CELL
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import pandas_datareader as web

import matplotlib.pyplot as plt
from matplotlib import rc

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

#Get the stock quote 
#df = web.DataReader('AAPL', data_source='yahoo', start='2000-01-01', end='2020-10-15')
df = web.DataReader('AMZN', data_source='yahoo', start='2006-01-01', end='2018-01-01') 

#Show the data 
df


,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2006-01-03,47.849998,46.250000,47.470001,47.580002,7582200,47.580002
2006-01-04,47.730000,46.689999,47.490002,47.250000,7440900,47.250000
2006-01-05,48.200001,47.110001,47.160000,47.650002,5417200,47.650002
2006-01-06,48.580002,47.320000,47.970001,47.869999,6152900,47.869999
2006-01-09,47.099998,46.400002,46.549999,47.080002,8943100,47.080002
...,...,...,...,...,...,...
2017-12-22,1174.619995,1167.829956,1172.079956,1168.359985,1585100,1168.359985
2017-12-26,1178.319946,1160.550049,1168.359985,1176.760010,2005200,1176.760010
2017-12-27,1187.290039,1175.609985,1179.910034,1182.260010,1867200,1182.260010


In [ ]:
torch.cuda.is_available()


False

In [ ]:
torch.cuda.current_device()

RuntimeError: ignored

In [ ]:
torch.cuda.get_device_name(0) # Get name device with ID '0'

In [ ]:
torch.cuda.memory_allocated()

In [ ]:
torch.cuda.memory_reserved()

In [ ]:
df.shape

In [ ]:
#Visualize the closing price history
plt.figure(figsize=(16,8))
plt.title('Close Price History')
plt.plot(df['Close'])
plt.xlabel('Date',fontsize=18)
plt.ylabel('Close Price USD ($)',fontsize=18)
plt.show()

In [ ]:
#Create a new dataframe with only the 'Close' column
data = df.filter(['Close'])

In [ ]:
data.head()

# Prepare the data
For the first set of exercises we'll
* divide the data into train and test sets
* normalize the training set
* prepare windowed seq/label tuples for an LSTM model

## 1. Divide the data into train and test sets
Working with a window_size of 12, divide the dataset into a sequence of 313 training records (including the window), and a test set of 12 records.

## 2. Normalize the training set
Feature scale the training set to fit within the range [-1,1].

In [ ]:
y = data.values.astype(float)

In [ ]:
train_size = int(len(y) * 0.8) 
test_size = len(y) - train_size
window_size = 60


In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
data_norm = scaler.fit_transform(y.reshape(-1,1))

In [ ]:
train_set = data_norm[:train_size]
test_set = data_norm[train_size:]


In [ ]:
# Run the code below to check your results:
print(f'Train: {len(train_set)}')
print(f'Test:  {len(test_set)}')
print(f'Train Shape: {train_set.shape}')
print(f'Test Shape:  {test_set.shape}')

In [ ]:
# DON'T WRITE HERE

In [ ]:
# Run the code below to check your results:
print(f'First item, original: {y[0]}')
print(f'First item, scaled:  {train_set[0]}')

In [ ]:
# DON'T WRITE HERE

## 3. Prepare data for LSTM
Prepare the list of windowed sequence/label tuples to be fed into an LSTM model.

In [ ]:
# RUN THIS CELL

def create_dataset_train(X,ws=1):
  Xs, ys = [], []
  for i in range (len(X) - ws):
    v = X[i:i+ws]
    Xs.append(v)
    ys.append(X[i+ws:i+ws+1,0])
  return np.array(Xs), np.array(ys)

x_train, y_train = create_dataset_train(train_set, window_size)

In [ ]:
test_set_2 = data_norm[train_size - window_size:]

def create_dataset_test(X,ws=1):
  Xs, ys = [], []
  for i in range (len(X) - ws):
    v = X[i:i+ws]
    Xs.append(v)
    ys.append(X[i+ws:i+ws+1,0])
  return np.array(Xs), np.array(ys)

x_test, y_test = create_dataset_test(test_set_2, window_size)

In [ ]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
x_train = torch.from_numpy(x_train).type(torch.Tensor).cuda()
x_test = torch.from_numpy(x_test).type(torch.Tensor).cuda()
y_train_lstm = torch.from_numpy(y_train).type(torch.Tensor).cuda()
y_test_lstm = torch.from_numpy(y_test).type(torch.Tensor).cuda()

## 4. Define the model
Design a model that has a (1,64) LSTM layer and a (64,1) fully-connected linear layer. Be sure to initialize $h_0$ and $c_0$, and return only the last predicted value.

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_().cuda()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_().cuda()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [ ]:
model = LSTM(input_dim=1, hidden_dim=512, output_dim=1, num_layers=1).cuda()
criterion = torch.nn.MSELoss()
optimiser = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
next(model.parameters()).is_cuda

In [ ]:
for m in model.parameters():
    print(m.device) #return cuda:0

In [ ]:
import time
num_epochs = 400
hist = np.zeros(num_epochs)
hist_val = np.zeros(num_epochs)
start_time = time.time()

for t in range(num_epochs):
    model.train()
    y_train_pred = model(x_train)

    loss = criterion(y_train_pred, y_train_lstm)
    hist[t] = loss.item()

    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

    model.eval()
    with torch.no_grad():
      y_test_pred = model(x_test)
      val_loss = criterion(y_test_pred, y_test_lstm)
      hist_val[t] = val_loss.item()
    
    if t % 10 == 0 and t !=0:
      print("Epoch ", t, "Train Loss: {}  Val Loss: {}".format(loss.item(), val_loss.item()))

training_time = time.time()-start_time
print("Training time: {}".format(training_time))

In [ ]:
plt.plot(hist)
plt.plot(hist_val)

## 10. Inverse transform the train and predicted values
Rescale the predicted values up to the original test set range.

In [ ]:
# invert predictions
y_train_pred = scaler.inverse_transform(y_train_pred.detach().cpu().numpy())
y_train = scaler.inverse_transform(y_train_lstm.detach().cpu().numpy())
y_test_pred = scaler.inverse_transform(y_test_pred.detach().cpu().numpy())
y_test = scaler.inverse_transform(y_test_lstm.detach().cpu().numpy())

In [ ]:
train_data = data[:train_size-window_size]
train_data["Train Predict"] = y_train_pred
valid = data[train_size:]
valid["Test"] = y_test 
valid['True Predictions'] = y_test_pred

In [ ]:
train_data

In [ ]:
valid

## BONUS EXERCISE: Plot the result
Plot the true_predictions values together with the original data. Remember to create a range of datetime values for the predicted data.

In [ ]:
train_data.plot(figsize=(18,10));

In [ ]:
valid.plot(figsize=(18,10));

In [ ]:
data.plot(figsize=(22,10))
#train_data['Train_Predict'].plot()
valid['True Predictions'].plot()
train_data["Train Predict"].plot()
plt.legend()

In [ ]:
# calculate root mean squared error of the training result based on last iteration (epoch) and non-scaled values.

trainScore = math.sqrt(mean_squared_error(y_train_pred, y_train))
print('Train Score: %.4f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test_pred, y_test))
print('Test Score: %.4f RMSE' % (testScore))

## Great job!